#Лабораторная работа **1**

[Задание](https://artemiikabanov.notion.site/1-80e692e92f6240d7af807db090400bb6): Найти набор данных и провести классификацию методом k- ближайших соседей.

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, mean_squared_error
from sklearn import svm
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
!pip install catboost
from catboost import CatBoostClassifier
from sklearn.naive_bayes import GaussianNB

In [4]:
data = pd.read_csv('data/Realistic_Medical_Dataset.csv')

print(f"Количество строк: {data.shape[0]}")
print(f"Количество столбцов: {data.shape[1]}")
print("Типы данных:")
print(data.dtypes)

Количество строк: 10000
Количество столбцов: 15
Типы данных:
Name                  object
Gender                object
Diagnosis             object
Description           object
Doctor                object
Hospital              object
Department            object
Start Date            object
End Date              object
Blood Type            object
Cholesterol Level    float64
Blood Sugar Level    float64
Pulse                  int64
Pressure              object
Complaints            object
dtype: object


In [5]:
scaler = StandardScaler()
label_encoder = LabelEncoder()
model = KNeighborsClassifier()

# target - Diagonesn, поэтому убираем из датасета этот параметр
remove_data = data.drop(['Diagnosis'], axis=1)

# Преобразование данных
for column in remove_data.columns:
    if remove_data[column].dtype == 'object':
        remove_data[column] = label_encoder.fit_transform(remove_data[column])

# Обучение Scarler и стандартизация признаков
scaler.fit(remove_data)
scaled_features = scaler.transform(remove_data)
scaled_data = pd.DataFrame(scaled_features, columns = remove_data.columns)

# Разделение датасета на обучающиеся и тестовые данные
x = scaled_data
y = data['Diagnosis']
x_train_data, x_test_data, y_train_data, y_test_data = train_test_split(x, y, test_size=0.3, random_state=42)

# Настройка параметров
param_grid = {'n_neighbors': list(range(2, 11)), 'metric': ['euclidean', 'manhattan', 'chebyshev', 'minkowski']}

# Создание объекта GridSearch с кросс-валидацией
grid_search = GridSearchCV(model, param_grid, cv=5)

# Обучение GridSearch для поиска лучших параметров
grid_search.fit(x_train_data, y_train_data)

# Получение лучших параметров и модели с этими параметрами
best_params = grid_search.best_params_
best_knn = grid_search.best_estimator_

# Получение имен из тестового набора данных
names_test = data.loc[x_test_data.index]['Name'].values

# Предсказания на тестовой выборке с использованием лучшей модели
predictions = best_knn.predict(x_test_data)

# Оценка точности лучшей модели
print(f"\nЛучшие параметры: {best_params}")

# Создание и вывод отчета и точности
print(classification_report(y_test_data, predictions))
print(accuracy_score(y_test_data, predictions))


Лучшие параметры: {'metric': 'manhattan', 'n_neighbors': 10}
              precision    recall  f1-score   support

      Asthma       0.92      0.97      0.94       738
  Depression       0.93      0.97      0.95       754
    Diabetes       0.90      0.88      0.89       758
Hypertension       0.92      0.85      0.89       750

    accuracy                           0.92      3000
   macro avg       0.92      0.92      0.92      3000
weighted avg       0.92      0.92      0.92      3000

0.917


# Лабораторная работа 2

[Задание](https://artemiikabanov.notion.site/2-ffac675dbf1f408a9f23e71889d70d61): Провести классификацию dataset методом машины опорных векторов.

In [6]:
data = pd.read_csv('data/Realistic_Medical_Dataset.csv')

print(f"Количество строк: {data.shape[0]}")
print(f"Количество столбцов: {data.shape[1]}")
print("Типы данных:")
print(data.dtypes)

Количество строк: 10000
Количество столбцов: 15
Типы данных:
Name                  object
Gender                object
Diagnosis             object
Description           object
Doctor                object
Hospital              object
Department            object
Start Date            object
End Date              object
Blood Type            object
Cholesterol Level    float64
Blood Sugar Level    float64
Pulse                  int64
Pressure              object
Complaints            object
dtype: object


In [7]:
scaler = StandardScaler()
label_encoder = LabelEncoder()
model = svm.SVC(kernel='poly', degree=2)

# target - Diagonesn, поэтому убираем из датасета этот параметр
new_data = data.drop(['Diagnosis'], axis=1)

# Преобразование данных
for column in new_data.columns:
    if new_data[column].dtype == 'object':
        new_data[column] = label_encoder.fit_transform(new_data[column])

# Обучение scaler и стандартизация признаков
scaler.fit(new_data)
scaled_features = scaler.transform(new_data)
scaled_data = pd.DataFrame(scaled_features, columns = new_data.columns)

# Разделение датасета на обучающиеся и тестовые данные
x = scaled_data
y = data['Diagnosis']
x_train_data, x_test_data, y_train_data, y_test_data = train_test_split(x, y, test_size=0.3, random_state=42)

# Указание гиперпараметров и их значений для поиска
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'sigmoid'],
    'gamma': [0.1, 1, 10]}

# Создание объекта GridSearch
grid_search = GridSearchCV(model, param_grid, cv=5)

# Обучение GridSearch
grid_search.fit(x_train_data, y_train_data)

# Получение лучших параметров и модели
best_params = grid_search.best_params_
best_svm = grid_search.best_estimator_

print(f'Лучшие параметры: {best_params}')

# Создание прогнозов с использованием лучшей модели
y_pred = best_svm.predict(x_test_data)

# Создание и вывод отчета и точности
print(classification_report(y_test_data, y_pred, zero_division=1))
print(accuracy_score(y_test_data, predictions))

Лучшие параметры: {'C': 0.01, 'gamma': 0.1, 'kernel': 'linear'}
              precision    recall  f1-score   support

      Asthma       1.00      1.00      1.00       738
  Depression       1.00      1.00      1.00       754
    Diabetes       1.00      1.00      1.00       758
Hypertension       1.00      1.00      1.00       750

    accuracy                           1.00      3000
   macro avg       1.00      1.00      1.00      3000
weighted avg       1.00      1.00      1.00      3000

0.917


# Лабораторная работа 3

[Задание](https://artemiikabanov.notion.site/3-683008e8b83f4926b099cc4efb64e600): Провести классификацию dataset методами линейной и логической регрессий.

In [8]:
data = pd.read_csv('data/Realistic_Medical_Dataset.csv')

print(f"Количество строк: {data.shape[0]}")
print(f"Количество столбцов: {data.shape[1]}")
print("Типы данных:")
print(data.dtypes)

Количество строк: 10000
Количество столбцов: 15
Типы данных:
Name                  object
Gender                object
Diagnosis             object
Description           object
Doctor                object
Hospital              object
Department            object
Start Date            object
End Date              object
Blood Type            object
Cholesterol Level    float64
Blood Sugar Level    float64
Pulse                  int64
Pressure              object
Complaints            object
dtype: object


In [9]:
scaler_lin = StandardScaler()
scaler_log = StandardScaler()
model_lin = LinearRegression()
model_log = LogisticRegression(penalty='l2', C=1.0, solver='lbfgs', max_iter=1000)

# Преобразование данных
for column in new_data.columns:
    if data[column].dtype == 'object':
        data[column] = label_encoder.fit_transform(data[column])

# Линейная регрессия для предсказания уровня холестерина на основе других признаков, а именно пол, пульс, давление, уровень сахара в крови
# Предварительная обработка данных для линейной регрессии
features_lin = ['Gender', 'Pulse', 'Pressure', 'Blood Sugar Level']
target_lin = 'Cholesterol Level'

# Разделение данных на признаки и целевую переменную
X_lin = data[features_lin]
y_lin = data[target_lin]

# Разделение на обучающую и тестовую выборку
X_train_lin, X_test_lin, y_train_lin, y_test_lin = train_test_split(X_lin, y_lin, test_size=0.3, random_state=42)

# Масштабирование числовых данных
X_train_lin = scaler_lin.fit_transform(X_train_lin)
X_test_lin = scaler_lin.transform(X_test_lin)

# Обучение модели линейной регрессии
model_lin.fit(X_train_lin, y_train_lin)

# Предсказание и оценка модели линейной регрессии
y_pred_lin = model_lin.predict(X_test_lin)
mse_lin = mean_squared_error(y_test_lin, y_pred_lin)

# Среднеквадратичная ошибка
print(mse_lin)

# Логистическая регрессия для классификации наличия гипертонии на основе медицинских показателей
# Предварительная обработка данных для логистической регрессии
data['Is_Hypertension'] = data['Diagnosis'].apply(lambda x: 1 if x == 'Hypertension' else 0)

features_log = ['Gender', 'Cholesterol Level', 'Blood Sugar Level', 'Pulse', 'Pressure']
target_log = 'Is_Hypertension'

X_log = data[features_log]
y_log = data[target_log]

# Разделение на обучающую и тестовую выборку
X_train_log, X_test_log, y_train_log, y_test_log = train_test_split(X_log, y_log, test_size=0.3, random_state=42)

# Масштабирование числовых данных
X_train_log = scaler_log.fit_transform(X_train_log)
X_test_log = scaler_log.transform(X_test_log)

# Обучение модели логистической регрессии
model_log.fit(X_train_log, y_train_log)

# Предсказание и оценка модели логистической регрессии
y_pred_log = model_log.predict(X_test_log)
accuracy_log = accuracy_score(y_test_log, y_pred_log)

# Вывод отчета
print(classification_report(y_test_log, y_pred_log, zero_division=1))
# Вывод точности
print(accuracy_log)

664.3682414385165
              precision    recall  f1-score   support

           0       0.75      1.00      0.86      2250
           1       1.00      0.00      0.00       750

    accuracy                           0.75      3000
   macro avg       0.88      0.50      0.43      3000
weighted avg       0.81      0.75      0.64      3000

0.75


# Лабораторная работа 4

[Задание](https://artemiikabanov.notion.site/4-2cfb349132074514a002749efac0bf8d): Провести классификацию dataset методами наивного Байеса.

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from sklearn.naive_bayes import GaussianNB

In [11]:
data = pd.read_csv('data/Realistic_Medical_Dataset.csv')

print(f"Количество строк: {data.shape[0]}")
print(f"Количество столбцов: {data.shape[1]}")
print("Типы данных:")
print(data.dtypes)

Количество строк: 10000
Количество столбцов: 15
Типы данных:
Name                  object
Gender                object
Diagnosis             object
Description           object
Doctor                object
Hospital              object
Department            object
Start Date            object
End Date              object
Blood Type            object
Cholesterol Level    float64
Blood Sugar Level    float64
Pulse                  int64
Pressure              object
Complaints            object
dtype: object


In [12]:
label_encoder = LabelEncoder()
model = GaussianNB()

# Удаление ненужных признаков
remove_data = data.drop(['Name', 'Hospital', 'Department', 'Start Date', 'End Date'], axis=1)

# Преобразование данных
for column in remove_data.columns:
    if remove_data[column].dtype == 'object':
        remove_data[column] = label_encoder.fit_transform(remove_data[column])

# Разделение датасета на обучающиеся и тестовые данные
x = remove_data.drop(['Diagnosis'], axis=1)
y = remove_data['Diagnosis']
x_train_data, x_test_data, y_train_data, y_test_data = train_test_split(x, y, test_size=0.3, random_state=42)

# Обучение модели методом опорных векторов
model.fit(x_train_data, y_train_data)

# Предсказание
predictions = model.predict(x_test_data)

# Создание и вывод отчета
print(classification_report(y_test_data, predictions))
# Создание и вывод точности
print(accuracy_score(y_test_data, predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       738
           1       1.00      1.00      1.00       754
           2       1.00      1.00      1.00       758
           3       1.00      1.00      1.00       750

    accuracy                           1.00      3000
   macro avg       1.00      1.00      1.00      3000
weighted avg       1.00      1.00      1.00      3000

1.0


# Лабораторная работа 5

[Задание](https://artemiikabanov.notion.site/5-c9a3114f5ba342b298e53f59ec4edcab): Провести классификацию dataset методами решающего дерева и случайного леса.

In [13]:
data = pd.read_csv('data/Realistic_Medical_Dataset.csv')

print(f"Количество строк: {data.shape[0]}")
print(f"Количество столбцов: {data.shape[1]}")
print("Типы данных:")
print(data.dtypes)

Количество строк: 10000
Количество столбцов: 15
Типы данных:
Name                  object
Gender                object
Diagnosis             object
Description           object
Doctor                object
Hospital              object
Department            object
Start Date            object
End Date              object
Blood Type            object
Cholesterol Level    float64
Blood Sugar Level    float64
Pulse                  int64
Pressure              object
Complaints            object
dtype: object


In [15]:
label_encoder = LabelEncoder()
decision_tree_model = DecisionTreeClassifier(random_state=42)
random_forest_model = RandomForestClassifier()

# Преобразование данных
for column in data.columns:
    if data[column].dtype == 'object':
        data[column] = label_encoder.fit_transform(data[column])

# Кодирование категориального признака
data['Diagnosis'] = label_encoder.fit_transform(data['Diagnosis'])

# Выбор признаков (медицинские параметры)
features = ['Cholesterol Level', 'Blood Sugar Level', 'Pulse', 'Pressure']

X = data[features]
y = data['Diagnosis']

# Разделение данных на обучающую и тестовую выборку
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Настройка гиперпараметров для решающего дерева
params_dt = {
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
# Настройка гиперпараметров для случайного дерева
params_rf = {
    'n_estimators': [50, 100],
    'max_depth': [None, 5],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1]
}

#Создание и обучение GridSearch для решающего дерева
grid_search_dt = GridSearchCV(decision_tree_model, params_dt, cv=5)
grid_search_dt.fit(X_train, y_train)
best_decision_tree = grid_search_dt.best_estimator_

#Создание и обучение GridSearch для случайного дерева
grid_search_rf = GridSearchCV(random_forest_model, params_rf, cv=5)
grid_search_rf.fit(X_train, y_train)
best_random_forest = grid_search_rf.best_estimator_

# Предсказание и оценка точности для решающего дерева
y_pred_tree = best_decision_tree.predict(X_test)
accuracy_tree = accuracy_score(y_test, y_pred_tree)
print("Лучшие параметры для решающего дерева:", grid_search_dt.best_params_)
print(classification_report(y_test, y_pred_tree))
print(accuracy_tree)

# Предсказание и оценка точности для случайного леса
y_pred_forest = best_random_forest.predict(X_test)
accuracy_forest = accuracy_score(y_test, y_pred_forest)
print("\nЛушчие параметры для случайного леса:", grid_search_rf.best_params_)
print(classification_report(y_test, y_pred_forest))
print(accuracy_forest)


Лучшие параметры для решающего дерева: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2}
              precision    recall  f1-score   support

           0       0.24      0.25      0.25       738
           1       0.26      0.20      0.23       754
           2       0.24      0.16      0.19       758
           3       0.24      0.36      0.29       750

    accuracy                           0.24      3000
   macro avg       0.24      0.24      0.24      3000
weighted avg       0.24      0.24      0.24      3000

0.24333333333333335

Лушчие параметры для случайного леса: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}
              precision    recall  f1-score   support

           0       0.23      0.26      0.24       738
           1       0.27      0.27      0.27       754
           2       0.26      0.25      0.25       758
           3       0.26      0.24      0.25       750

    accuracy                           0.26

# Лабораторная работа 6

[Задание](https://artemiikabanov.notion.site/6-5cd85ebfdbd3460cae3feb81cd219a6c): Провести классификацию dataset методами CatBoost .

In [16]:
data = pd.read_csv('data/Realistic_Medical_Dataset.csv')

print(f"Количество строк: {data.shape[0]}")
print(f"Количество столбцов: {data.shape[1]}")
print("Типы данных:")
print(data.dtypes)

Количество строк: 10000
Количество столбцов: 15
Типы данных:
Name                  object
Gender                object
Diagnosis             object
Description           object
Doctor                object
Hospital              object
Department            object
Start Date            object
End Date              object
Blood Type            object
Cholesterol Level    float64
Blood Sugar Level    float64
Pulse                  int64
Pressure              object
Complaints            object
dtype: object


In [17]:
label_encoder = LabelEncoder()
catboost_model = CatBoostClassifier(verbose=0)

# Преобразование данных
for column in data.columns:
    if data[column].dtype == 'object':
        data[column] = label_encoder.fit_transform(data[column])

# Выбор признаков (медицинские параметры)
features = ['Cholesterol Level', 'Blood Sugar Level', 'Pulse', 'Pressure']

X = data[features]
y = data['Diagnosis']

# Разделение данных на обучающую и тестовую выборку
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Определение параметров для поиска
param_grid = {
    'iterations': [100, 200,],
    'learning_rate': [0.01, 0.1],
    'depth': [3, 6],
    'loss_function': ['MultiClass']
}

# Создание объекта GridSearchCV для модели
grid_search = GridSearchCV(
    estimator=CatBoostClassifier(verbose=False),
    param_grid=param_grid,
    cv=5,
    n_jobs=-1
)

# Подгонка модели к данным
grid_search.fit(X_train, y_train)

# Получение лучших параметров и оценщика с лучшими параметрами
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

print(f'Лучшие параметры: {best_params}')

# Получение прогнозов с использованием лучшей модели
y_pred = best_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(classification_report(y_test, y_pred, zero_division=1))
print(accuracy)

Лучшие параметры: {'depth': 3, 'iterations': 200, 'learning_rate': 0.01, 'loss_function': 'MultiClass'}
              precision    recall  f1-score   support

           0       0.25      0.42      0.31       738
           1       0.25      0.15      0.19       754
           2       0.25      0.17      0.20       758
           3       0.25      0.28      0.26       750

    accuracy                           0.25      3000
   macro avg       0.25      0.25      0.24      3000
weighted avg       0.25      0.25      0.24      3000

0.251
